In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchsummary import summary
import json
import random
import pandas as pd
from pathlib import Path

import argparse

from src.utils.configer import Configer

from src.models.customResNet import customResNet
from src.models.customUNet import customUNet
from src.models.customResNetUNet import customResNetUNet

In [2]:
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version used by PyTorch: {torch.version.cuda}")
else:
    print("CUDA is not available. PyTorch is using CPU.")

CUDA is available!
Number of GPUs: 1
Current GPU Name: NVIDIA GeForce RTX 3060
CUDA Version used by PyTorch: 12.6


In [3]:
parser = argparse.ArgumentParser()

parser.add_argument('--disable-cuda', action='store_true',
                    help='Disable CUDA')
parser.add_argument('--hypes', default=None, type=str,
                    dest='hypes', help='The file of the hyper parameters.')
parser.add_argument('--phase', default='train', type=str,
                    dest='phase', help='The phase of module.')
parser.add_argument('--gpu', default=[0, ], nargs='+', type=int,
                    dest='gpu', help='The gpu used.')
parser.add_argument('--resume', default=None, type=str,
                    dest='resume', help='The path of pretrained model.')

args, unknown = parser.parse_known_args()
args.hypes = 'src/hyperparameters/customResNet-config.json'

torch.autograd.set_detect_anomaly(True)
configer = Configer(args)

# Read config-files.
hyperparameters_dir = Path("./src/hyperparameters/")

config_path = hyperparameters_dir / "config.json"
assert config_path.exists(), f"Config not found: {config_path}"
with open(config_path, "r") as f:
    configer.general_config = json.load(f)

model_config_path = hyperparameters_dir / f"{configer.get('model_name')}-config.json"
assert model_config_path.exists(), f"Config not found: {model_config_path}"
with open(model_config_path, "r") as f:
    configer.model_config = json.load(f)

dataset_config_path = hyperparameters_dir / f"{configer.get('dataset_name')}-config.json"
assert dataset_config_path.exists(), f"Config not found: {dataset_config_path}"
with open(dataset_config_path, "r") as f:
    configer.dataset_config = json.load(f)
    
configer.device = torch.device(configer.general_config.get("device").lower() if torch.cuda.is_available() else 'cpu')


In [4]:
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True  # To have ~deterministic results
        torch.backends.cudnn.benchmark = False

set_seed(configer.general_config.get("seed"))

In [5]:
selected_classes = configer.dataset_config.get("selected_classes")
n_classes = len(selected_classes )

mdl_input_size = configer.model_config.get('input_size')

model = customResNet(
        layers_config = configer.model_config.get("layers_num")*[configer.model_config.get("block_size")],
        in_channels = mdl_input_size[0],
        layer0_channels = configer.model_config.get("output_channels") // 2**(configer.model_config.get("layers_num") - 1),
        num_classes = n_classes
        )
model = model.to(configer.device)

test_input = torch.randn(1, *mdl_input_size)
print(configer.model_config.get("layers_num")*[configer.model_config.get("block_size")])
model_size = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model size: {model_size}")

summary(model.to(configer.device), tuple(mdl_input_size))

[2, 2, 2, 2]
Model size: 889732
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 18, 64, 64]           2,646
       BatchNorm2d-2           [-1, 18, 64, 64]              36
              ReLU-3           [-1, 18, 64, 64]               0
         MaxPool2d-4           [-1, 18, 64, 64]               0
            Conv2d-5           [-1, 18, 32, 32]             324
       BatchNorm2d-6           [-1, 18, 32, 32]              36
            Conv2d-7           [-1, 18, 32, 32]           2,916
       BatchNorm2d-8           [-1, 18, 32, 32]              36
              ReLU-9           [-1, 18, 32, 32]               0
           Conv2d-10           [-1, 18, 32, 32]           2,916
      BatchNorm2d-11           [-1, 18, 32, 32]              36
             ReLU-12           [-1, 18, 32, 32]               0
       BasicBlock-13           [-1, 18, 32, 32]               0
       

In [ ]:
model = customUNet(
    in_channels=3,
    out_channels=1,
    features=[18, 36, 72, 144]
    )
model = model.to(configer.device)

test_input = torch.randn(1, 3, 128, 128).to(configer.device)
test_output = model(test_input)

print(f"   Вход:  {test_input.shape}")
print(f"   Выход: {test_output.shape}")

Encoder features by level: [18, 36, 72, 144]
Input: torch.Size([1, 3, 128, 128])
Encoder: torch.Size([1, 18, 128, 128])
Pool: torch.Size([1, 18, 64, 64])
Input: torch.Size([1, 18, 64, 64])
Encoder: torch.Size([1, 36, 64, 64])
Pool: torch.Size([1, 36, 32, 32])
Input: torch.Size([1, 36, 32, 32])
Encoder: torch.Size([1, 72, 32, 32])
Pool: torch.Size([1, 72, 16, 16])
Input: torch.Size([1, 72, 16, 16])
Encoder: torch.Size([1, 144, 16, 16])
Pool: torch.Size([1, 144, 8, 8])
   Вход:  torch.Size([1, 3, 128, 128])
   Выход: torch.Size([1, 1, 128, 128])


In [7]:
model = customResNetUNet(
    in_channels=3,
    out_channels=1,
    features=[18, 36, 72, 144],
    backbone_layers_config = 4*[2],
    backbone_layer0_channels = 18,
    backbone_pretrained = False,
    backbone_checkpoints_path = "_",
    device = configer.device
    )
model = model.to(configer.device)

test_input = torch.randn(1, 3, 128, 128).to(configer.device)
test_output = model(test_input)

print(f"   Вход:  {test_input.shape}")
print(f"   Выход: {test_output.shape}")

Encoder features by level: [18, 36, 72, 144]
Input: torch.Size([1, 3, 128, 128])
Encoder: torch.Size([1, 18, 64, 64])
Input: torch.Size([1, 18, 64, 64])
Encoder: torch.Size([1, 36, 32, 32])
Input: torch.Size([1, 36, 32, 32])
Encoder: torch.Size([1, 72, 16, 16])
Input: torch.Size([1, 72, 16, 16])
Encoder: torch.Size([1, 144, 8, 8])
   Вход:  torch.Size([1, 3, 128, 128])
   Выход: torch.Size([1, 1, 128, 128])
